# 1.Load Dataset

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import os

data_path = "C:\\kuliah\\semester 4\\KAL\\dataset\\asl_dataset\\asl_dataset"
output_path = "C:\\kuliah\\semester 4\\KAL\\dataset\\output"

categories = {  0: "0",
                1: "1",
                2: "2",
                3: "3",
                4: "4",
                5: "5",
                6: "6",
                7: "7",
                8: "8",
                9: "9",
                10: "a",
                11: "b",
                12: "c",
                13: "d",
                14: "e",
                15: "f",
                16: "g",
                17: "h",
                18: "i",
                19: "j",
                20: "k",
                21: "l",
                22: "m",
                23: "n",
                24: "o",
                25: "p",
                26: "q",
                27: "r",
                28: "s",
                29: "t",
                30: "u",
                31: "v",
                32: "w",
                33: "x",
                34: "y",
                35: "z",
            }

# 2.Preprocessing

In [4]:

import splitfolders
splitfolders.ratio(data_path, output=output_path, seed=1333, ratio=(0.8, 0.1, 0.1))


datagen = ImageDataGenerator(rescale=1.0 / 255)
batch_size, img_size = 32, 200

def load_data(path):
    return datagen.flow_from_directory(path, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')

train_data = load_data(output_path + "\\train")
val_data = load_data(output_path + "\\val")
test_data = load_data(output_path + "\\test")






Copying files: 2515 files [00:02, 917.89 files/s]

Found 2012 images belonging to 36 classes.


Found 251 images belonging to 36 classes.
Found 252 images belonging to 36 classes.


# 3.Membangun Arsitektur CNN

In [6]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(36, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




# 4.Training Model

In [7]:
model.fit(train_data, validation_data=val_data, epochs=15, callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, factor=0.5)
])




Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 62s 960ms/step - accuracy: 0.2653 - loss: 2.8595 - val_accuracy: 0.8645 - val_loss: 0.4005 - learning_rate: 0.0010
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 47s 746ms/step - accuracy: 0.8307 - loss: 0.5521 - val_accuracy: 0.9283 - val_loss: 0.1953 - learning_rate: 0.0010
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 47s 743ms/step - accuracy: 0.9177 - loss: 0.2434 - val_accuracy: 0.9283 - val_loss: 0.1715 - learning_rate: 0.0010
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 51s 816ms/step - accuracy: 0.9436 - loss: 0.1701 - val_accuracy: 0.9402 - val_loss: 0.1839 - learning_rate: 0.0010
Epoch 5/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 57s 900ms/step - accuracy: 0.9614 - loss: 0.1236 - val_accuracy: 0.9402 - val_loss: 0.1648 - learning_rate: 0.0010
Epoch 6/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 56s 882ms/step - accuracy: 0.9640 - loss: 0.1053 - val_accuracy: 0.9641 - val_loss: 0.1311 - learning_rate: 0.0010
Epoch 7/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 54s 853ms/step - accuracy: 0.9818 - loss: 0.

# 5.Evaluasi Model

In [9]:
test_loss, test_acc = model.evaluate(test_data, verbose=2)
print(f'Akurasi Model: {test_acc:.4f}')

c:\Users\zamr\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 - 4s - 447ms/step - accuracy: 0.9484 - loss: 0.2306
Akurasi Model: 0.9484


# 6.Prediksi Model 

In [ ]:

def predict_user_image(image_path):
    img = load_img(image_path, target_size=(img_size, img_size))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = np.argmax(model.predict(img_array), axis=1)[0]
    class_label = categories[prediction]  # Get actual ASL character
    print(f'Predicted Class: {prediction} ({class_label})')
    return class_label


image_path = "hand1_a_dif_seg_5_cropped.jpeg"
predict_user_image(image_path)